Begin to implement chains and implement a pipeline 

In [1]:
with open("job_desc.md") as f:
    description = f.read()
with open("resume.md") as f:
    resume = f.read()
title = 'django developer'

In [2]:
from util import list_job_requirements, chunk_markdown, emulate_section_wording

In [3]:
# requirements = list_job_requirements(description)

In [4]:

from util import cut_sections

sections = cut_sections(resume)

In [5]:
history = sections['history']['result']
chunks = chunk_markdown(history)

We need a function to only select job history from chunks. Any other headings such as summary, or education should be set aside.

## Begin to process job history chunks

In [7]:
for chunk in chunks:
    print(chunk, '\n\n\n')
    # emulated = emulate_section_wording(title, description, chunk)
    # TODO: place emphasis on any occurrences of job requirements
    # TODO: format section
    # print(emulated)

page_content='Responsible for maintaining electrical and instrument systems for 11 F-18E aircraft totaling $715 million. Additionally, responsible for final check and troubleshooting of aircraft before flight.  \n- Assisted primary supervisor in management of maintenance programs leading to no hits on a major inspection.\n- Expertly managed tool program of over $175,000 by training coworkers on expectations and proper procedures leading to no hits across 2 major and 3 routine inspections.\n- Awarded for showing expertise and initiative for reinstalling an incorrectly installed part under stressful conditions.\n- Lead and mentored a group of 3 individuals in flight deck operations and proper flight deck cleaning procedures as supervisor leading to no accidents and rejuvenating a stagnant work center.' metadata={'Header 1': 'Experience', 'Header 2': 'Aviation Electrician, US Navy, Aug 2020 - Present (2 years 11 months)'} 



page_content="Passionate about leveraging technology to drive b